In [4]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import os
import datetime
import pandas as pd
from IPython.display import display

In [5]:
from base64 import b64decode
from io import BytesIO

from IPython import get_ipython
from IPython.core.magic import register_cell_magic
import PIL


@register_cell_magic
def capture_png(line, cell):
    get_ipython().run_cell_magic(
        'capture',
        ' --no-stderr --no-stdout result',
        cell
    )
    out_paths = line.strip().split(' ')
    for output in result.outputs:
        data = output.data
        if 'image/png' in data:
            path = out_paths.pop(0)
            if not path:
                raise ValueError('Too few paths given!')
            png_bytes = data['image/png']
            if isinstance(png_bytes, str):
                png_bytes = b64decode(png_bytes)
            assert isinstance(png_bytes, bytes)
            bytes_io = BytesIO(png_bytes)
            image = PIL.Image.open(bytes_io)
            image.save(path, 'png')

In [6]:
#Read in Precip data files

# root_dir = '/Users/meghan/Lemhi_Main/Lemhi/CONUS404/CONUS404_Data/C404_area_avgs'
root_dir = '/Users/stewartl/sb/ncar/Lemhi_Precip'
LRidge = nc.Dataset(os.path.join(root_dir, 'lemhiridge.nc'),'r')
BLower =  nc.Dataset(os.path.join(root_dir, 'beaverheadlower.nc'),'r')

In [7]:
LRidge_Time = LRidge.variables['Time'][:]
LRidge_AS_LWC = LRidge.variables['AS_LWC'][:]
LRidge_Precip = LRidge.variables['PREC_ACC_NC'][:]
LRidge_Temp = LRidge.variables['AS_Tc'][:]

BLower_Time = BLower.variables['Time'][:]
BLower_AS_LWC = BLower.variables['AS_LWC'][:]
BLower_Precip = BLower.variables['PREC_ACC_NC'][:]
BLower_Temp = BLower.variables['AS_Tc'][:]

In [14]:
#Lemhi Ridge Precip with Beaverhead Lower Criteria
precip_df = pd.DataFrame({'time':LRidge_Time, 'precip' :LRidge_Precip})
criteria_df = pd.DataFrame({'time':BLower_Time, 'temp':BLower_Temp,
                             'lwc':BLower_AS_LWC})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit1_df = _crit_df[_crit_df['temp'] > -18]
    _crit2_df = _crit1_df[_crit1_df['lwc'] > 0.01]
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Seedable Precip':(_crit2_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
display(df)
df.describe()




Total Precip  Seedable Precip
Region           Precip      Year                               
Beaverhead Lower Lemhi Ridge 1980    359.786163       155.488818
                             1981    354.554413       141.676714
                             1982    398.460148       125.004118
                             1983    383.033126       137.222669
                             1984    374.637762       108.345915
                             1985    288.656402        75.335975
                             1986    342.474767       145.983313
                             1987    295.165928        82.192369
                             1988    436.635696       152.530082
                             1989    322.695001       120.858570
                             1990    284.451934        77.449436
                             1991    396.559401       183.888068
                             1992    276.176659       118.196620
                             1993    323.872079       105.935518
                             1994    305.827162       102.266791
                             1995    455.760206       237.301166
                             1996    464.736755       173.540117
                             1997    433.576148       173.290903
                             1998    426.556779       165.958712
                             1999    381.654663       163.818874
                             2000    290.102102        85.843985
                             2001    284.317591        87.976245
                             2002    252.344298        97.632436
                             2003    397.471532       181.608921
                             2004    277.513247        84.798537
                             2005    305.484303       111.508116
                             2006    379.279525       110.328938
                             2007    344.810499       122.427326
                             2008    325.511473        81.997613
                             2009    379.654685       103.861245
                             2010    392.016207       147.404341
                             2011    405.006573       190.609499
                             2012    353.645199       181.872162
                             2013    298.872214       129.153481
                             2014    528.150069       279.454119
                             2015    338.891098       149.915387
                             2016    369.324410       168.550593
                             2017    437.801136       224.680920
                             2018    443.259070       230.210055
                             2019    281.576157       115.899941
                             2020    430.250167       170.382358
                             2021    230.397927        46.404281

,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,358.355968,139.257268
std,66.227821,49.554494
min,230.397927,46.404281
25%,300.525237,104.379813
50%,357.170288,133.188075
75%,398.212994,169.924417
max,528.150069,279.454119
